# Segmenting and Clustering Neighborhoods in Toronto

## 1. Scrape Data from Wikipedia page and creat a dataframe

#### Scrape Data from Wikipedia page

In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import numpy as np
import pandas as pd

In [3]:
html = urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").read()

In [4]:
soup = BeautifulSoup(html, features='lxml') ###read with lxml format
type(soup)

bs4.BeautifulSoup

In [5]:
table = soup.find('table','wikitable sortable')
table0 = table.find_all('tr')[0].find_all('th')[0].get_text() # use lxml read header of table
table0

'Postcode'

In [6]:
table1 = table.find_all('tr')[1].find_all('td')[2].get_text()# use lxml read body of table
table1

'Not assigned\n'

#### Read table infor into a list table_NB

In [7]:
table_NB = []
for i,text in enumerate(table.find_all('tr')):
    if i == 0:
        for j in [0,1,2]:
            table_NB.append(table.find_all('tr')[i].find_all('th')[j].get_text().replace('\n',''))
    else:
        brgh = table.find_all('tr')[i].find_all('td')[1].get_text().replace('\n','')
        if brgh != 'Not assigned': # not reading 'Not assigned' infor
            for j in [0,1,2]:
                table_NB.append(table.find_all('tr')[i].find_all('td')[j].get_text().replace('\n',''))

        

#### Format table_NB into pd.DataFrame

In [8]:
table_NB = np.array(table_NB)
table_NB = table_NB.reshape(-1,3)
table_NBpd = pd.DataFrame(table_NB)

In [9]:
columns = table_NBpd.loc[0,]
table_NBpd.columns = columns

In [10]:
table_NBpd = table_NBpd.drop([0])

In [11]:
table_NBpd = table_NBpd.reset_index(drop=True)
table_NBpd.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [12]:
table_NBpd.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object', name=0)

#### Find 'Not assigned' Neighbourhood and use Borough's value

In [13]:
a = table_NBpd.loc[table_NBpd['Neighbourhood']=='Not assigned']

In [14]:
table_NBpd.at[a.index[0],'Neighbourhood']=table_NBpd.at[a.index[0],'Borough']

In [15]:
table_final = table_NBpd.sort_values('Postcode').groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: "%s" % ','.join(x)).reset_index()
table_final.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
table_final.shape

(103, 3)

## 2. Add Latitude and longitude

In [17]:
latlog_pd = pd.read_csv('Geospatial_Coordinates.csv')

In [18]:
latlog_pd.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
table_final = table_final.merge(latlog_pd.sort_values('Postal Code'),left_on ='Postcode',right_on = 'Postal Code').drop(['Postal Code'],axis = 1)

In [20]:
table_final.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## 3. Explore and cluster Toronto area

In [216]:
table_toronto = table_final[table_final['Borough'].str.contains('Toronto')].reset_index(drop=True) # only cluster borough with 'Toronto' in name

In [217]:
table_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [193]:
import json
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

#### use Foursquare to explore venues in Boroughs in toronto

In [189]:
CLIENT_ID = 'EMDHRUDTJF4AQ2T1PTM5GUFKORZA0DM5VY4RBARSILADEC1S' # your Foursquare ID
CLIENT_SECRET = 'NGFPELYJNTRZJUODY3HOT3MIVW0DISNNJQFSI4QZ0RI3NHEC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EMDHRUDTJF4AQ2T1PTM5GUFKORZA0DM5VY4RBARSILADEC1S
CLIENT_SECRET:NGFPELYJNTRZJUODY3HOT3MIVW0DISNNJQFSI4QZ0RI3NHEC


In [218]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [219]:
toronto_venues = getNearbyVenues(names=table_toronto['Neighbourhood'],
                                   latitudes=table_toronto['Latitude'],
                                   longitudes=table_toronto['Longitude']
                                  )

#### Preparation for clustering and cluster based on most poplular venus category

In [220]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West,Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [221]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_cluster = toronto_grouped.drop(['Neighbourhood'],axis=1)

In [222]:
kclusters = 5
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_cluster)
# check cluster labels generated for each row in the dataframe
label = kmeans.labels_ 

In [223]:
toronto_cluster['Neighbourhood']=toronto_grouped['Neighbourhood']
toronto_cluster['label']=label
toronto_cluster.head()

,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Neighbourhood,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.030000,0.0,...,0.010000,0.0,0.0,0.010000,0.0,0.0,0.01,0.000000,"Adelaide,Richmond,King",3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,Berczy Park,3
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.058824,Business Reply Mail Processing Centre 969 Eastern,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,"Cabbagetown,St. James Town",3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012346,0.0,...,0.012346,0.0,0.0,0.012346,0.0,0.0,0.00,0.012346,Central Bay Street,3


In [224]:
toronto_NLL = toronto_venues.loc[:,['Neighbourhood','Neighborhood Latitude','Neighborhood Longitude']].drop_duplicates().sort_values('Neighbourhood')
toronto_NLL.shape

(38, 3)

In [225]:
toronto_merged = pd.DataFrame(toronto_cluster.loc[:,['label','Neighbourhood']])
toronto_merged.shape

(38, 2)

In [226]:
toronto_merged = toronto_merged.join(toronto_NLL.set_index('Neighbourhood'),on='Neighbourhood')
toronto_merged.head()

,label,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude
0,3,"Adelaide,Richmond,King",43.650571,-79.384568
1,3,Berczy Park,43.644771,-79.373306
2,3,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
3,3,"Cabbagetown,St. James Town",43.667967,-79.367675
4,3,Central Bay Street,43.657952,-79.387383


#### view clusters in map view

In [228]:
latitude = toronto_merged['Neighborhood Latitude'][0]
longitude = toronto_merged['Neighborhood Longitude'][0]
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Neighborhood Latitude'], toronto_merged['Neighborhood Longitude'], toronto_merged['Neighbourhood'], toronto_merged['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### let view what are most populated venue category in those clusters

In [229]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 5
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide,Richmond,King",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Café,Steakhouse,Bakery
2,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Garden,Pizza Place,Recording Studio
3,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Pizza Place,Café,Italian Restaurant
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Bubble Tea Shop


In [230]:
toronto_pp = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'),on = 'Neighbourhood').drop(['Neighborhood Latitude','Neighborhood Longitude'],1)
toronto_pp.head()

,label,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,3,"Adelaide,Richmond,King",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant
1,3,Berczy Park,Coffee Shop,Cocktail Bar,Café,Steakhouse,Bakery
2,3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Garden,Pizza Place,Recording Studio
3,3,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Pizza Place,Café,Italian Restaurant
4,3,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Bubble Tea Shop


#### Cluster 0

In [231]:
toronto_0 = toronto_pp.loc[toronto_pp['label']==0]
toronto_0

,label,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
19,0,"Moore Park,Summerhill East",Playground,Gym,Tennis Court,Diner,Fast Food Restaurant
32,0,The Beaches,Neighborhood,Health Food Store,Coffee Shop,Pub,Yoga Studio


#### Cluster 1

In [232]:
toronto_1 = toronto_pp.loc[toronto_pp['label']==1]
toronto_1

,label,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
25,1,Roselawn,Garden,Yoga Studio,Fish Market,Filipino Restaurant,Fast Food Restaurant


#### Cluster 2

In [233]:
toronto_2 = toronto_pp.loc[toronto_pp['label']==2]
toronto_2

,label,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
18,2,Lawrence Park,Park,Swim School,Bus Line,Yoga Studio,Dog Run


#### Cluster 3

In [234]:
toronto_3 = toronto_pp.loc[toronto_pp['label']==3]
toronto_3

,label,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,3,"Adelaide,Richmond,King",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant
1,3,Berczy Park,Coffee Shop,Cocktail Bar,Café,Steakhouse,Bakery
2,3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Garden,Pizza Place,Recording Studio
3,3,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Pizza Place,Café,Italian Restaurant
4,3,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Bubble Tea Shop
5,3,Christie,Grocery Store,Café,Park,Athletics & Sports,Nightclub
6,3,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant
7,3,"Commerce Court,Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant
8,3,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Italian Restaurant
9,3,Davisville North,Hotel,Burger Joint,Gym,Breakfast Spot,Park


#### Cluster 4

In [235]:
toronto_4 = toronto_pp.loc[toronto_pp['label']==4]
toronto_4

,label,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
24,4,Rosedale,Park,Playground,Trail,Diner,Fast Food Restaurant


## Conclusion: 
This assignment part3 clusters Toronto borough into 5 clusters based on populate venues' category. Data is retrieved from Foursquare website and K-mean cluster is used to cluster borough. 
From clustering results, most of area in Toronto fell in to cluster 3, which I call it 'Food area'. From mapview, this 'Food area' covers most part of Toronto, which means Toronto is a city with many populations and large amount of food traffic everyday, otherwise, I won't see so many food shops.Cluster 0 looks like a residential area with playgroud and even 'Neighborhood' label on it. Other 3 clusters have only 1 data for each, while it is hard to generalize those neighbourhood's feature, I guess those neighbourhood are more for outdoor or indoor workwout. 